In [1]:
import keras


Using TensorFlow backend.


In [2]:
keras.__version__

'1.2.2'

In [3]:
%matplotlib inline
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *

/home/jaidmin/anaconda2/envs/tensorflow/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
import keras_tqdm
path = "./data/cats-dogs/"

In [5]:
parms = {'verbose': 0, 'callbacks': [TQDMNotebookCallback(leave_inner=True)]}

In [6]:
rn_mean = np.array([123.68,116.779,1003.939], dtype=np.float32).reshape((1,1,3))
inp = Input((224,224,3))
preproc = Lambda(lambda x: (x -rn_mean))(inp) # do we need to reverse the color channels ?? the resnet paper only says to substract the pixel mean

In [7]:
??ResNet50

In [8]:
model = ResNet50(include_top=False, input_tensor=preproc)

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 112, 112, 64)  9472        zeropadding2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
res5b_branch2a = model.get_layer("res5b_branch2a") # we want the layer right before this one (like jeremy explained)
last_conv_layer = model.layers[model.layers.index(res5b_branch2a) - 1 ].output

In [11]:
conv_model = Model(inp, last_conv_layer)

In [12]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 112, 112, 64)  9472        zeropadding2d_1[0][0]            
___________________________________________________________________________________________

In [13]:
avg_pool = AveragePooling2D((7,7))

In [14]:
conv_model_avg_pooling = Model(inp, Flatten()(avg_pool(conv_model.output)))

In [15]:
conv_model_avg_pooling.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 112, 112, 64)  9472        zeropadding2d_1[0][0]            
___________________________________________________________________________________________

In [16]:
for layer in conv_model_avg_pooling.layers:
    layer.trainable = False

### data preparation

dont execute more than one time

In [ ]:
# create validation set
from shutil import move
import glob
import os


In [ ]:
# structure training data for keras
all_images_paths = glob.glob(path + "train/*")
all_images_filenames = [pathname.split("/")[-1] for pathname in all_images_paths]

if not os.path.exists(path + "train/dogs"):
    os.makedirs(path + "train/dogs")
if not os.path.exists(path + "train/cats"):    
    os.makedirs(path + "train/cats")

dogs = [filename for filename in all_images_filenames if "dog" in filename]
cats = [filename for filename in all_images_filenames if "cat" in filename]

for dog in dogs:
    move(path + "train/" + dog, path + "train/dogs/" + dog)

for cat in cats:
    move(path + "train/" + cat, path + "train/cats/" + cat)

In [ ]:
from random import shuffle

In [ ]:
if not os.path.exists(path + "valid"):
    os.makedirs(path + "valid")
    os.makedirs(path + "valid/cats")
    os.makedirs(path + "valid/dogs")

shuffle(dogs)
shuffle(cats)

for dog in dogs[:int(len(dogs) * 0.2)]:
    move(path + "train/dogs/"+ dog, path + "valid/dogs/" + dog)
    
for cat in cats[:int(len(cats) * 0.2)]:
    move(path + "train/cats/"+ cat, path + "valid/cats/" + cat)

### training

In [17]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size, )

In [18]:
trn_batches = get_batches(path + "train", shuffle=False)
val_batches = get_batches(path + "valid", shuffle=False)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# precompute convolutional output

trn_conv_features = conv_model_avg_pooling.predict_generator(trn_batches, trn_batches.nb_sample)
val_conv_features = conv_model_avg_pooling.predict_generator(val_batches,val_batches.nb_sample)

In [ ]:
if not os.path.exists(path + "results"):
    os.makedirs(path+ "results")
save_array(path + "results/trn_conv_features", trn_conv_features)
save_array(path + "results/val_conv_features", val_conv_features)

In [19]:
trn_conv_features = load_array(path + "results/trn_conv_features")
val_conv_features = load_array(path + "results/val_conv_features")

In [20]:
from keras.utils.np_utils import to_categorical
trn_labels = to_categorical(trn_batches.classes)
val_labels = to_categorical(val_batches.classes)

In [21]:
fully_connected_model = Sequential([
    Dense(input_shape =(2048,),output_dim=2,activation="softmax"),   
])

In [22]:
fully_connected_model.compile(Adam(lr=1e-3),"binary_crossentropy", metrics = ["accuracy"])

In [23]:
fully_connected_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 2)             4098        dense_input_1[0][0]              
Total params: 4,098
Trainable params: 4,098
Non-trainable params: 0
____________________________________________________________________________________________________


In [24]:
fully_connected_model.fit(trn_conv_features, trn_labels,nb_epoch=20, **parms)

In [ ]:
fully_connected_model = Sequential([
    Dense(input_shape =(2048,),output_dim=512,activation="relu"),
    Dense(output_dim=2,activation="softmax")
])

In [ ]:
fully_connected_model.compile(Adam(lr=1e-4),"categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
fully_connected_model.fit(trn_conv_features, trn_labels,nb_epoch=15, **parms)